In [1]:
# importing the required modules  
import json
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaConsumer
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt
# import matplotlib
# matplotlib.use('Qt5Agg')



In [2]:
# Kafka Consumer  
trade_consumer = KafkaConsumer(  
    'successful_trade',  
     bootstrap_servers = ['localhost : 9092'],
     value_deserializer = lambda x : loads(x.decode('utf-8'))  
)



In [ ]:
df = pd.DataFrame()

for message in trade_consumer:
    df_iter = pd.json_normalize(message.value)
    df_iter = df_iter.drop(["id", "id_sell", "quantity"], axis=1)
    df_iter['update_timestamp'] = pd.to_datetime(df_iter['update_timestamp'])
    df = df.append(
        df_iter, 
        ignore_index=True
    )
    resampled_data = df.groupby('instrument').resample(
        '2T', on='update_timestamp'
    ).last().drop(
        ["instrument", "update_timestamp"], axis=1
    )
    window_size = 4
    resampled_data["sma"] = resampled_data.groupby(["instrument"])["price"].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).mean()
    )
    output_path = "outputs/sma_calc.csv"
    resampled_data.to_csv(
            output_path, 
            header=True,
        )